# Using Graphs and GNN to model time series

## Time Series Forecasting with Graph Convolutional Neural Networks

Time Series forecasting tasks can be carried out following different approaches. The most classical is based on statistical and autoregressive methods. More tricky are the algorithms based on boosting and ensemble where we have to produce a good amount of useful handmade features with rolling periods. On the other side, we can find neural network models that enable more freedom in their development, providing customizable adoption of sequential modeling and much more.

Recurrent and convolutional structure achieve great success in time series forecasting. Interesting approaches in the field are given by the adoption of Transformers and Attention architectures, originally native in the NLP. Uncommon seems to be the usage of graph structures, where we have a network composed of different nodes that are related by some kind of linkage to each other. What we try to do is to use a graphical representation of our time series to produce future forecasts.

In this post, we carry out a sales forecasting task where we make use of graph convolutional neural networks exploiting the nested structure of our data, composed of different sales series of various items in different stores.

In [ ]:
import os
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from datetime import date, timedelta
from scipy.stats import skew, kurtosis

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras import backend as K

### IMPORT SPEKTRAL CLASSES ###

from spektral_utilities import *
from spektral_gcn import GraphConv

### The Data

The dataset is collected form a padast competition on Kaggle. The store item demand forecasting challenge, provides 4 whole years of sales data in a daily format for different items sold in various stores. We have 10 stores and 50 products, for a total of 500 series. Each product is sold in every store. Our scope is to provide accurate future forecast daily for all the items.   

In [ ]:
### READ DATA ###

df = pd.read_csv('sales_train.csv.zip')
df['date'] = pd.to_datetime(df['date'])

print(df.shape)
df.head()

In [ ]:
### SWITCH DATA FROM VERTICAL TO HORIZONTAL FORMAT ###

unstaked_df = df.copy()
unstaked_df['id'] = df['item'].astype(str)+'_'+df['store'].astype(str)
unstaked_df.set_index(['id','date'], inplace=True)
unstaked_df.drop(['store','item'], axis=1, inplace=True)
unstaked_df = unstaked_df.astype(float).unstack()
unstaked_df.columns = unstaked_df.columns.get_level_values(1)

print(unstaked_df.shape)
unstaked_df.head()

In [ ]:
### UTILITY FUNCTIONS FOR FEATURE ENGINEERING ###

sequence_length = 14



def get_timespan(df, today, days):    
    df = df[pd.date_range(today - timedelta(days=days), 
            periods=days, freq='D')] # day - n_days <= dates < day    
    return df

def create_features(df, today, seq_len):
    
    all_sequence = get_timespan(df, today, seq_len).values
    
    group_store = all_sequence.reshape((-1, 10, seq_len))
    
    store_corr = np.stack([np.corrcoef(i) for i in group_store], axis=0)
    
    store_features = np.stack([
              group_store.mean(axis=2),
              group_store[:,:,int(sequence_length/2):].mean(axis=2),
              group_store.std(axis=2),
              group_store[:,:,int(sequence_length/2):].std(axis=2),
              skew(group_store, axis=2),
              kurtosis(group_store, axis=2),
              np.apply_along_axis(lambda x: np.polyfit(np.arange(0, sequence_length), x, 1)[0], 2, group_store)
            ], axis=1)
    
    group_store = np.transpose(group_store, (0,2,1))
    store_features = np.transpose(store_features, (0,2,1))
    
    return group_store, store_corr, store_features

def create_label(df, today):
    
    y = df[today].values
    
    return y.reshape((-1, 10))

In [ ]:
### PLOT A SEQUENCE OF SALES FOR ITEM 10 IN ALL STORES ###

sequence = get_timespan(unstaked_df, date(2017,11,1), 30)
sequence.head(10).T.plot(figsize=(14,5))
plt.ylabel('sales')
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.5))

In [ ]:
### DEFINE TRAIN, VALID, TEST DATES ###

train_date = date(2013, 1, 1)
valid_date = date(2015, 1, 1)
test_date = date(2016, 1, 1)

The data at our disposal is minimal: only sales amount and numerical encoding of items and stores. This is still enough for us to underline a basic hierarchical structure. All we need to do is to group the series at item levels, in this way we end with 50 groups (items) each composed by 10 series (items sold in each store); an example of a group is depicted in the figure above.

In classical graph networks, all the relevant information is stored in an object called the adjacent matrix. This is a numerical representation of all the linkages present in the data. The adjacent matrix in our context can be retrieved by the correlation matrix calculated on sale sequences of a given item in all stores.

The sequence repartition is fundamental in our approach because we decide to process the data in pieces like for recurrent architecture, which will be also part of our model.

In [ ]:
### CREATE VALID FEATURES ###

X_seq, X_cor, X_feat, y = [], [], [], []

for d in tqdm(pd.date_range(valid_date+timedelta(days=sequence_length), test_date)):
    seq_, corr_, feat_ = create_features(unstaked_df, d, sequence_length)
    y_ = create_label(unstaked_df, d)
    X_seq.append(seq_), X_cor.append(corr_), X_feat.append(feat_), y.append(y_)
    
X_valid_seq = np.concatenate(X_seq, axis=0).astype('float16')
X_valid_cor = np.concatenate(X_cor, axis=0).astype('float16')
X_valid_feat = np.concatenate(X_feat, axis=0).astype('float16')
y_valid = np.concatenate(y, axis=0).astype('float16')

print(X_valid_seq.shape, X_valid_cor.shape, X_valid_feat.shape, y_valid.shape)

In [ ]:
### CREATE TEST FEATURES ###

X_seq, X_cor, X_feat, y = [], [], [], []

for d in tqdm(pd.date_range(test_date+timedelta(days=sequence_length), date(2016,12,31))):
    seq_, corr_, feat_ = create_features(unstaked_df, d, sequence_length)
    y_ = create_label(unstaked_df, d)
    X_seq.append(seq_), X_cor.append(corr_), X_feat.append(feat_), y.append(y_)
    
X_test_seq = np.concatenate(X_seq, axis=0).astype('float16')
X_test_cor = np.concatenate(X_cor, axis=0).astype('float16')
X_test_feat = np.concatenate(X_feat, axis=0).astype('float16')
y_test = np.concatenate(y, axis=0).astype('float16')

print(X_test_seq.shape, X_test_cor.shape, X_test_feat.shape, y_test.shape)

In [ ]:
### SCALE SEQUENCES ###

scaler_seq = StandardScaler()
scaler_feat = StandardScaler()

X_train_seq = scaler_seq.fit_transform(X_train_seq.reshape(-1,10)).reshape(X_train_seq.shape)
X_valid_seq = scaler_seq.transform(X_valid_seq.reshape(-1,10)).reshape(X_valid_seq.shape)
X_test_seq = scaler_seq.transform(X_test_seq.reshape(-1,10)).reshape(X_test_seq.shape)

y_train = scaler_seq.transform(y_train)
y_valid = scaler_seq.transform(y_valid)
y_test = scaler_seq.transform(y_test)

X_train_feat = scaler_feat.fit_transform(X_train_feat.reshape(-1,10)).reshape(X_train_feat.shape)
X_valid_feat = scaler_feat.transform(X_valid_feat.reshape(-1,10)).reshape(X_valid_feat.shape)
X_test_feat = scaler_feat.transform(X_test_feat.reshape(-1,10)).reshape(X_test_feat.shape)

In [ ]:
### OBTAIN LAPLACIANS FROM CORRELATIONS ###

X_train_lap = localpooling_filter(1 - np.abs(X_train_cor))
X_valid_lap = localpooling_filter(1 - np.abs(X_valid_cor))
X_test_lap = localpooling_filter(1 - np.abs(X_test_cor))

### The Model

Our model receives, as input, sequences of sales from all stores and adjacent matrixes obtained from the same sequences. The sequences are passed through LSTM layers, while the correlation matrixes are processed by GraphConvolution layers. They are implemented in Spektral, a cool library for graph deep learning build on Tensorflow. It has various kinds of graph layers available. We use the most basic one, the GraphConvolution. It operates a series of convolution operations between learnable weights, external node features (provided together with the adjacent matrix), and our correlation matrixes. Unlikely, at the moment Spektral doesn’t support Window so I have to extract manually the class of my interest and create my Python executable.

In [ ]:
def set_seed(seed):
    
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    random.seed(seed)


def get_model():
    
    set_seed(33)

    opt = Adam(lr=0.001)

    inp_seq = Input((sequence_length, 10))
    inp_lap = Input((10, 10))
    inp_feat = Input((10, X_train_feat.shape[-1]))

    x = GraphConv(32, activation='relu')([inp_feat, inp_lap])
    x = GraphConv(16, activation='relu')([x, inp_lap])
    x = Flatten()(x)

    xx = LSTM(128, activation='relu', return_sequences=True)(inp_seq)
    xx = LSTM(32, activation='relu')(xx)

    x = Concatenate()([x,xx])
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation='relu')(x)
    x = Dense(32, activation='relu')(x)
    x = Dropout(0.3)(x)
    out = Dense(1)(x)

    model = Model([inp_seq, inp_lap, inp_feat], out)
    model.compile(optimizer=opt, loss='mse', 
                  metrics=[tf.keras.metrics.RootMeanSquaredError()])

    return model

In [ ]:
model = get_model()
model.summary() 

In [ ]:
### TRAIN A MODEL FOR EACH STORES USING ALL THE DATA AVAILALBE FROM OTHER STORES ###

pred_valid_all = np.zeros(y_valid.shape)
pred_test_all = np.zeros(y_test.shape)

for store in range(10):

    print('-------', 'store', store, '-------')
    
    es = EarlyStopping(patience=5, verbose=1, min_delta=0.001, monitor='val_loss', mode='auto', restore_best_weights=True)

    model = get_model()
    model.fit([X_train_seq, X_train_lap, X_train_feat], y_train[:,store], epochs=100, batch_size=256, 
              validation_data=([X_valid_seq, X_valid_lap, X_valid_feat], y_test[:,store]), callbacks=[es], verbose=2)

    pred_valid_all[:,store] = model.predict([X_valid_seq, X_valid_lap, X_valid_feat]).ravel()
    pred_test_all[:,store] = model.predict([X_test_seq, X_test_lap, X_test_feat]).ravel()


pred_valid_all = scaler_seq.inverse_transform(pred_valid_all)
reverse_valid = scaler_seq.inverse_transform(y_valid)
pred_test_all = scaler_seq.inverse_transform(pred_test_all)
reverse_test = scaler_seq.inverse_transform(y_test)

The train is computed with the first two years of data while the remaining two are respectively used for validation and testing. I trained a model for each store so we ended with a total of 10 different neural networks.

The predictions of stores are retrieved at the end of the training procedure by the relative models. The errors are calculated as RMSE on test data and reported below.

In [ ]:
### RMSE ON TEST DATA ###

error = {}

for store in range(10):
    
    error[store] = np.sqrt(mean_squared_error(reverse_test[:,store], pred_test_all[:,store]))

In [ ]:
### PLOT RMSE ###

plt.figure(figsize=(14,5))
plt.bar(range(10), error.values())
plt.xticks(range(10), ['store_'+str(s) for s in range(10)])
plt.ylabel('error')
np.set_printoptions(False)

In [ ]:
### UTILITY FUNCTION TO PLOT PREDICTION ###

def plot_predictions(y_true, y_pred, store, item):
    
    y_true = y_true.reshape(50,-1,10)
    y_pred = y_pred.reshape(50,-1,10)
    
    plt.plot(y_true[item,:,store], label='true')
    plt.plot(y_pred[item,:,store], label='prediction')
    plt.title(f"store: {store} item: {item}"); plt.legend()
    plt.ylabel('sales'); plt.xlabel('date')

In the same way, it’s easy to extract the predictions for items in desired stores directing manipulating our nested data structure.

In [ ]:
plt.figure(figsize=(11,5))
plot_predictions(reverse_test, pred_test_all, 7,0)

### Anomaly Detection in Multivariate Time Series in Networks Graphs

When working on an anomaly detection task, we are used to discovering and pointing out situations where the data register unseen dynamics. The ability to study the past and extrapolate a “normal” behavior is crucial for the success of most anomaly detection applications. In this situation, an adequate learning strategy must take into consideration the temporal dependency. What in the past may be considered “anomalous”, now may be marked as “normal” since the underlying data dynamics are prone to change.

A good multivariate anomaly detection system should study the historical relationship between data and alert possible future divergencies. Given these premises, the PCA (Principal Component Analysis) algorithm reveals to be a very good candidate for the task. Together with the old but gold PCA, we can leverage the connected nature of the signals at our disposal to develop a network structure and detect anomalies with a graph-based approach.

In this post, we carry out a multivariate anomaly detection task adopting an unsupervised approach based on network clustering. We build a network graph from a bunch of time series looking at their correlation. Then we apply, on top of the correlation matrix, the DBSCAN algorithm to identify potential anomalous patterns.

### The Data

We imagine working in a multivariate system. All the series show a positive correlation degree between each other while maintaining stationery for the whole period of reference.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import scipy.cluster.hierarchy as sch

from sklearn.cluster import DBSCAN
from sklearn.decomposition import PCA
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit

We manually insert in our data an anomalous period. We change a sequential bunch of observations, in a predefined feature, with some gaussian noise. We do this “swapping” operation taking care to maintain the same mean and standard deviation of the original data. In this way, the anomalous period is not visible simply by looking at the feature distributions. In other words, we alter the correlation present in the data, in a given time range, by inserting some gaussian noise that is not related to anything in the entire system.

Our scope is to develop an anomaly detection solution that can correctly point out the changes in feature relationships. Let's start adopting an approach based on PCA.

In [ ]:
### SYNTHETIC CORRELATED DATA GENERATION ###

np.random.seed(33)

n_features = 50
n_samples = 4_000

r = np.random.randint(-2,10, (n_features,n_features))
r = np.dot(r, r.T)

x = np.random.normal(0,1, (n_features,n_samples))
c = np.linalg.cholesky(r)

X = pd.DataFrame(
    np.dot(c, x).T, columns=[f"feat_{c+1}" for c in range(n_features)]
)

corr_mat = pairwise_distances(X.T, metric='correlation')

X.shape, corr_mat.shape

In [ ]:
### PLOT SERIES DISTRIBUTIONS ###

plt.figure(figsize=(18,6))
plt.subplot(1,2,1)
X.plot(legend=False, xlabel='timesteps', ax=plt.gca())
plt.subplot(1,2,2)
X.plot(kind='density', legend=False, ax=plt.gca())

In [ ]:
### PLOT SERIES CORRELATIONS ###

plt.figure(figsize=(12, 10))
sns.heatmap(X.corr(), annot=False, cmap='bwr')
plt.title('correlation matrix')
plt.ylabel('series'); plt.xlabel('series')

In [ ]:
### MANUALLY INSERT ANOMALOUS PERIOD ###

n_samples_change = 400
_ = np.random.randint(0,n_samples-n_samples_change)

X.loc[np.arange(_, _+n_samples_change), 'feat_2'] = \
    np.random.normal(X['feat_2'].mean(), X['feat_2'].std(), (n_samples_change,))

### PCA Anomaly Detection

PCA fits particularly well in our scenario. This is not surprising since it’s widely adopted in a lot of industrial applications for anomaly detection. Its adaptability and flexibility make it the standard solution when needing to detect anomalies in a multivariate system.

Detecting anomalies with PCA is straightforward. We leverage the compression ability of PCA to reduce the original feature dimensionality of the data. In this way, we preserve only meaningful interactions while reducing the number of features and discharging the noise. The dimensionality reduction operation is fully reversible. We can come back to the original data shape effortlessly using only the learned relationships in the original set of data.

We can calculate the reconstruction error to see and summarize the goodness of the compression operation.

In [ ]:
### PCA ANOMALY DETECTION ###

rec_errors_samples = {}
rec_errors_features = {}

for i, (past_id,future_id) in enumerate(
    TimeSeriesSplit(10, test_size=300).split(X)
):
    
    scaler = StandardScaler()
    pca = PCA(0.7, random_state=33)
    pca.fit(scaler.fit_transform(X.iloc[past_id]))
    
    Xt = pca.inverse_transform(
        pca.transform(
            scaler.transform(X.iloc[future_id])
        )
    )
    rec_errors_samples[past_id[-1]] = \
        np.linalg.norm(scaler.transform(X.iloc[future_id]) - Xt, axis=1)
    rec_errors_features[past_id[-1]] = \
        np.linalg.norm(scaler.transform(X.iloc[future_id]) - Xt, axis=0)

In [ ]:
### PLOT PCA RECONSTRUCTION ERRORS ###

plt.figure(figsize=(18,6))
plt.subplot(1,2,1)
plt.plot(list(rec_errors_samples.keys()), 
         [np.mean(r) for r in rec_errors_samples.values()])
plt.ylabel('recontruction error'); plt.xlabel('timesteps')
plt.title('sample-wise recontruction')

plt.subplot(1,2,2)
for i in range(n_features):
    rec = []
    for r in rec_errors_features.values():
        rec.append(r[i])
    plt.plot(list(rec_errors_features.keys()), rec)
plt.ylabel('recontruction error'); plt.xlabel('timesteps')
plt.title('feature-wise recontruction')

plt.show()

We can see the reconstruction error as an anomaly score. A high magnitude of the reconstruction error means that a change in the data relationships has happened. We can compute it sample-wise to attribute an anomaly score to each sample. We can also compute it for each feature over time and observe which feature is affected by an anomaly behavior. In both cases, we follow a temporal validation strategy to simulate the real data flow in a time-dependent system.

In our simulation study, we see how the reconstruction error increase in correspondence with the artificial anomalous period (both samples and feature-wise).

In [ ]:
### COMPUTE HIERARCHICAL CLUSTERING ON CORRELATION MATRIX ###

d = sch.distance.pdist(corr_mat)
L = sch.linkage(d, method='ward')
ind = sch.fcluster(L, d.max(), 'distance')
dendrogram = sch.dendrogram(L, no_plot=True)

labels = dendrogram['leaves']
corr_mat_cluster = pairwise_distances(
    pd.concat([X.iloc[:,[i]] for i in labels], axis=1).T,
    metric='correlation'
)

plt.figure(figsize=(18,8))
plt.subplot(1,2,1)
dendrogram = sch.dendrogram(L, no_plot=False)
plt.title('dendrogram')
plt.ylabel('distance'); plt.xlabel('series')
plt.subplot(1,2,2)
plt.imshow(corr_mat_cluster, cmap='bwr')
plt.title('correlation matrix')
plt.ylabel('series'); plt.xlabel('series')
plt.xticks(range(n_features), labels, rotation=90)
plt.yticks(range(n_features), labels)
plt.show()

### BDSCAN Anomaly Detection

To develop our graph-based methodology, as a first step, we simply have to divide the data into temporal windows of the same size.

Then we should compute the correlation of the series in each time window. The correlation matrix is used as an approximation of the internal system dynamics and the relationships between series. Other similarity measures can work well like euclidean distance or dynamic time warping.

As a final step, we fit a DBSCAN algorithm on each similarity matrix. In a normal situation, we should expect all the series to belong to the same cluster. In case of anomalies, we expect to have multiple clusters of series.

In [ ]:
### DBSCAN ANOMALY DETECTION ###

network_ano = {}
dbscan = DBSCAN(eps=0.6, min_samples=1, metric="precomputed")

for i, (past_id,_) in enumerate(
    TimeSeriesSplit(10, test_size=300, max_train_size=300).split(X)
):
    
    preds = dbscan.fit_predict(
        pairwise_distances(X.iloc[past_id].T, metric='correlation')
    )
    if (preds > 0).any():
        ano_features = list(X.columns[np.where(preds > 0)[0]])
        network_ano[past_id[-1]] = ano_features
    else:
        network_ano[past_id[-1]] = None

Correlated series are supposed to move together. If one of them changes its direction, concerning the others, we can mark it as anomalous. To get the best from this methodology, it’s important to work with series that show the same correlation degrees. That it’s not always possible due to the complex nature of some systems. In these cases, a preliminary clustering should be made. A simple hierarchical clustering may be good to group the series according to their nature and enable DBSCAN to achieve the best performances.

In [ ]:
### PLOT DBSCAN DETECTED ANOMALIES ###

roll_corr = X.rolling(300).corr()

for ano_loc,ano in network_ano.items():
    if ano is not None:
        for ano_feat in network_ano[ano_loc]:
            roll_corr[ano_feat].unstack().plot(
                legend=False, figsize=(11,6),
                title=f"{ano_feat} rolling correlation",
                ylabel='correlation', xlabel='timesteps'
            )
            plt.axvline(ano_loc, linestyle='--', c='black')
            plt.show()

In the end, the entire procedure detects the change in the internal system dynamic in correspondence with the anomalous period.